In [2]:
from random import randrange
from math import pi
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import QasmSimulator

class UnknownQubit:
    def __init__(self):
        self.__theta = randrange(18000) / 18000 * pi  # Random angle for qubit rotation
        self.__available_qubits = 1000  # Initially, 1000 qubits are available
        self.__active_qubits = 0  # No active qubits initially
        print(self.__available_qubits, "qubits are created")
    
    def get_qubits(self, number_of_qubits=None):
        if number_of_qubits is None or not isinstance(number_of_qubits, int) or number_of_qubits < 1:
            print("ERROR: The method 'get_qubits' takes the number of qubit(s) as a positive integer, i.e., get_qubits(100).")
        elif number_of_qubits <= self.__available_qubits:
            # Use the appropriate number of qubits in the QuantumCircuit
            self.__qc = QuantumCircuit(number_of_qubits, number_of_qubits)  # Dynamically set number of qubits
            self.__qc.ry(2 * self.__theta, 0)  # Apply a rotation on the first qubit
            self.__active_qubits = number_of_qubits
            self.__available_qubits -= self.__active_qubits
            print("You have", number_of_qubits, "active qubits that are set to (cos(theta), sin(theta))")
            self.available_qubits()
        else:
            print("WARNING: You requested", number_of_qubits, "qubits, but there is not enough available qubits!")
            self.available_qubits()
            
    def measure_qubits(self):
        if self.__active_qubits > 0:
            self.__qc.measure(range(self.__active_qubits), range(self.__active_qubits))  # Measure all active qubits
            simulator = QasmSimulator()  # Use the QasmSimulator to run the circuit
            job = simulator.run(self.__qc, shots=1024)  # Set a reasonable number of shots for a better result
            result = job.result()  # Get the result of the simulation
            counts = result.get_counts(self.__qc)  # Get the counts of the measurement results
            print("Your", self.__active_qubits, "qubits are measured")
            print("Counts:", counts)
            self.__active_qubits = 0  # After measurement, qubits are destroyed
            return counts
        else:
            print("WARNING: There are no active qubits -- you might first execute 'get_qubits()' method.")
            self.available_qubits()
            
    def rotate_qubits(self, angle=None):
        if angle is None or not isinstance(angle, (float, int)):
            print("ERROR: The method 'rotate_qubits' takes a real-valued angle in radian as its parameter, i.e., rotate_qubits(1.2121).")
        elif self.__active_qubits > 0:
            self.__qc.ry(2 * angle, 0)  # Apply rotation on the first qubit
            print("Your active qubits are rotated by angle", angle, "in radian.")
        else:
            print("WARNING: There are no active qubits -- you might first execute 'get_qubits()' method.")
            self.available_qubits()    
    
    def compare_my_guess(self, my_angle):
        if my_angle is None or not isinstance(my_angle, (float, int)):
            print("ERROR: The method 'compare_my_guess' takes a real-valued angle in radian as your guessed angle, i.e., compare_my_guess(1.2121).")
        else:
            self.__available_qubits = 0  # Set available qubits to zero after comparison
            diff = abs(my_angle - self.__theta)
            print(self.__theta, "is the original angle.")
            print(my_angle, "is your guess.")
            print("The angle difference between the original theta and your guess is", diff / pi * 180, "degrees.")
            print("--> The number of available qubits is now zero, and so you cannot make any further experiments.")

    def available_qubits(self):
        print("--> The number of available unused qubit(s) is", self.__available_qubits)

# Test the class
qubit_instance = UnknownQubit()
qubit_instance.get_qubits(5)  # Get 5 qubits
qubit_instance.rotate_qubits(1.5708)  # Rotate by 90 degrees in radians
qubit_instance.measure_qubits()  # Measure the qubits
qubit_instance.compare_my_guess(1.5)  # Compare with a guessed angle


1000 qubits are created
You have 5 active qubits that are set to (cos(theta), sin(theta))
--> The number of available unused qubit(s) is 995
Your active qubits are rotated by angle 1.5708 in radian.
Your 5 qubits are measured
Counts: {'00000': 568, '00001': 456}
2.3087215345380994 is the original angle.
1.5 is your guess.
The angle difference between the original theta and your guess is 46.33633073037654 degrees.
--> The number of available qubits is now zero, and so you cannot make any further experiments.
